
# BPSK vs. "Quantum Shift Keying" (QSK) — Interactive Notebook

This notebook creates side‑by‑side visualizations and simple math models for:

* **BPSK** — classical binary phase‑shift keying
* **Quantum phase flips** — a conceptual analogue to Norseen's "Quantum Shift Keying" (QSK), implemented as a π rotation on a qubit state.

You can reuse the figures in lectures or your repo.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def newfig():
    plt.figure(figsize=(10,5))



## 1) Classical BPSK

We model a carrier \( c(t)=\cos(2\pi f_c t) \) and encode bit **0** as phase \(0\) and bit **1** as phase \( \pi \).
The modulated signal is:

\[
s(t) = A \cos(2\pi f_c t + \pi b(t)),
\]

where \( b(t)\in\{0,1\} \) is the bit stream.


In [ ]:

# Parameters
fc = 1.0           # carrier frequency (Hz)
A  = 1.0           # amplitude
T  = 10.0          # seconds
N  = 1000
t  = np.linspace(0, T, N)

# Example: single phase flip at t = T/2
phase = np.zeros_like(t)
phase[t >= T/2] = np.pi  # flip to represent a '1' after midpoint

carrier = A*np.cos(2*np.pi*fc*t)
bpsk    = A*np.cos(2*np.pi*fc*t + phase)

newfig()
plt.plot(t, carrier, ls='--', alpha=.6, label='Carrier')
plt.plot(t, bpsk, label='BPSK signal')
plt.axvline(T/2, color='r', ls=':', label='Phase flip (π)')
plt.xlabel('Time [s]'); plt.ylabel('Amplitude')
plt.title('BPSK: phase flip encodes a bit')
plt.legend(); plt.grid(True); plt.tight_layout()
plt.show()



## 2) Conceptual QSK: Qubit Phase Flip

A pure qubit state on the Bloch sphere is \( |\psi\rangle = \cos(\theta/2)|0\rangle + e^{i\phi}\sin(\theta/2)|1\rangle \).  
A **phase flip** is a rotation around the *Z*‑axis by \( \pi \):

\[
R_z(\pi) = e^{-i\pi Z/2} = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}.
\]

Applying \( R_z(\pi) \) multiplies the \( |1\rangle \) component by \(-1\), i.e., a relative phase of \( \pi \).
We visualize this as a trajectory on the equator (θ = π/2).


In [ ]:

# Bloch equator trajectory before/after a Z-rotation by π

theta = np.pi/2  # equator
phi   = np.linspace(0, np.pi, 100)  # sweep 0 .. π

# Bloch coords: x = sinθ cosφ, y = sinθ sinφ, z = cosθ
x = np.sin(theta)*np.cos(phi)
y = np.sin(theta)*np.sin(phi)

plt.figure(figsize=(10,4.8))
plt.plot(x, y, label='State precession (equator)', color='green')
plt.scatter([np.sin(theta)*np.cos(0)], [np.sin(theta)*np.sin(0)], color='blue', label='Start')
plt.scatter([np.sin(theta)*np.cos(np.pi)], [np.sin(theta)*np.sin(np.pi)], color='red', label='After $R_z(\\pi)$')
plt.axis('equal'); plt.grid(True)
plt.xlabel('Bloch X'); plt.ylabel('Bloch Y')
plt.title('Quantum phase flip as $R_z(\\pi)$ (conceptual QSK)')
plt.legend(); plt.tight_layout(); plt.show()



## 3) Optional: Binary Symbol Stream

We generate a short random bit stream, BPSK‑modulate it, and show the waveform plus a simple coherent correlator.


In [ ]:

rng = np.random.default_rng(7)
bits = rng.integers(0,2, size=12)

# Waveform parameters
fc = 2.0
Ts = 1.0     # symbol time (s)
Fs = 400     # samples per second
Ns = int(Ts*Fs)

# reference carrier
n  = np.arange(Ns)/Fs
ref0 = np.cos(2*np.pi*fc*n + 0.0)     # bit 0
ref1 = np.cos(2*np.pi*fc*n + np.pi)   # bit 1 (phase-flipped)

# synthesize
sig = []
for b in bits:
    phase = 0.0 if b==0 else np.pi
    sig.append(np.cos(2*np.pi*fc*n + phase))
sig = np.concatenate(sig)

# add noise
sigma = 0.2
rx = sig + rng.normal(0, sigma, size=sig.shape)

# coherent detection by correlation per symbol
dec = []
for k in range(len(bits)):
    seg = rx[k*Ns:(k+1)*Ns]
    c0 = np.dot(seg, ref0)
    c1 = np.dot(seg, ref1)
    dec.append(0 if c0>=c1 else 1)

# Plot a few symbols
newfig()
tt = np.arange(len(sig))/Fs
plt.plot(tt[:5*Ns], rx[:5*Ns], label='Received (noisy)')
plt.plot(tt[:5*Ns], sig[:5*Ns], alpha=.6, ls='--', label='Clean BPSK')
plt.xlabel('Time [s]'); plt.ylabel('Amplitude')
plt.title('BPSK symbol stream (first 5 symbols)')
plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

print("Bits:     ", bits.tolist())
print("Detected: ", dec)
print("BER:      ", np.mean(bits!=np.array(dec)))
